In [27]:
# Importing tensorflow

import tensorflow as tf

# Importing some more libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [28]:
DATASET_RATINGS_FILE = "filmAdvice/system/try_ratings.csv"

In [29]:
ratings = pd.read_csv(DATASET_RATINGS_FILE,\
          sep=",", engine='python')

In [31]:
print(ratings.shape)

(2000, 4)


In [34]:
ratings_pivot = pd.pivot_table(ratings[['userId','movieId','rating']], values='rating', index='userId', columns='movieId' ).fillna(0)
print(ratings_pivot)

movieId  1      2      3      5      6      7      10     11     14     15     \
userId                                                                          
1          0.0    3.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2          0.0    0.0    4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4          0.0    0.0    0.0    0.0    3.0    0.0    4.0    0.0    0.0    0.0   
5          0.0    3.0    0.0    0.0    0.0    0.0    0.0    5.0    0.0    0.0   
6          5.0    0.0    3.0    0.0    0.0    5.0    0.0    0.0    0.0    0.0   
7          0.0    0.0    3.0    0.0    0.0    3.0    0.0    4.0    0.0    2.0   
8          4.0    0.0    5.0    0.0    3.0    0.0    4.0    0.0    0.0    0.0   
9          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10         4.0    0.0    0.0    0.0    0.0    0.0    0.0    4.0    0.0    0.0   
11         4.5    0.0    0.0

In [36]:
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)
print(X_train)
print(X_test.head())

movieId  1      2      3      5      6      7      10     11     14     15     \
userId                                                                          
2          0.0    0.0    4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
12         4.0    0.0    3.0    2.0    3.0    3.0    0.0    0.0    0.0    0.0   
14         4.5    0.0    0.0    0.0    0.0    3.5    0.0    0.0    0.0    0.0   
1          0.0    3.5    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
8          4.0    0.0    5.0    0.0    3.0    0.0    4.0    0.0    0.0    0.0   
18         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
7          0.0    0.0    3.0    0.0    0.0    3.0    0.0    4.0    0.0    2.0   
4          0.0    0.0    0.0    0.0    3.0    0.0    4.0    0.0    0.0    0.0   
16         3.0    0.0    0.0    0.0    3.0    0.0    0.0    0.0    0.0    0.0   
17         0.0    0.0    0.0    0.0    4.0    0.0    0.0    0.0    0.0    0.0   
6          5.0    0.0    3.0

In [75]:
n_nodes_inpl = 1260  
n_nodes_hl1  = 100 
n_nodes_outl = 1260
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1, n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl])) }

In [81]:
input_layer = tf.placeholder('float', [None, 1260])
print(input_layer)
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
print(input_layer_const)
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,\
hidden_1_layer_vals['weights']))
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
output_true = tf.placeholder('float', [None, 1260])
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
learn_rate = 0.1
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
hm_epochs =200
tot_users = X_train.shape[0]
print(tot_users)

Tensor("Placeholder_12:0", shape=(?, 1260), dtype=float32)
Tensor("Fill_12:0", shape=(?, 1), dtype=float32)
14


In [82]:
sess = tf.Session()
sess.run(init)
batch_size = 100
hm_epochs = 50
tot_users = X_train.shape[0]
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_users/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, output_true: epoch_x})
        print("Debug4")
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 0 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 1 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 2 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 3 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 4 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 5 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 6 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 7 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 8 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 9 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 10 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 11 / 50 loss: 0
MSE train 49.06348065291772 MSE test 47.566801454065704
Epoch 12 / 50 loss

In [102]:
sample_user = X_test.iloc[11,:]
print(sample_user)

#get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})
print(sample_user_pred)

IndexError: single positional indexer is out-of-bounds